In [11]:
import torch.nn as nn 
import torch 

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.core = nn.Sequential(
            nn.Linear(4,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,2)
        )
    def forward(self,x):
        return self.core(x)


policy = Model()
value = Model()




In [12]:
import gymnasium as gym 

env = gym.make("CartPole-v1")
obs, info = env.reset(seed=42)
obs = torch.tensor(obs)

In [ ]:
def is_terminal(observation):
    x,v,theta , omega = observation 
    x_terminal = x <= -4.8 or x>=4.8
    theta_terminal = theta <= -0.209*7.5 or theta >= 0.209*7.5
    return x_terminal or theta_terminal


episodes = 100
for episode in range(episodes):
    logits = policy(obs)
    probs = torch.softmax(logits,dim=-1)
    action = torch.multinomial(probs, num_samples=1).item()
    obs, reward, terminated, truncated , info = env.step(action)
    obs = torch.tensor(obs)
    

    

TypeError: multinomial(): argument 'num_samples' (position 2) must be int, not Tensor